# Orbit games

We consider energy plots and orbital solutions in polar coordinates for the general potential energy

$\begin{align}
   U(r) = k r^n
\end{align}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.integrate import simps

In [ ]:
# Change the common font size
font_size = 14
plt.rcParams.update({'font.size': font_size})

In [2]:
class Orbit:
    """
    Potentials and associated differential equations for central force motion
    with the potential U(r) = k r^n.
    """
    
    def __init__(self, ang_mom, n, k=1, mu=1):
        self.ang_mom = ang_mom
        self.n = n
        self.k = k
        self.mu = mu
    
    def U(self, r):
        """Potential energy of the form U = kr^n."""
        return self.k * r**self.n
    
    def Ucf(self, r):
        """Centrifugal potential energy"""
        return self.ang_mom**2 / (2. * self.mu * r**2)
    
    def Ueff(self, r):
        """Effective potential energy"""
        return self.U(r) + self.Ucf(r)
    
    def U_deriv(self, r):
        """dU/dr"""
        return self.n * self.k * r**(self.n - 1)
        
    def Ucf_deriv(self, r):
        """dU_cf/dr"""
        return -2. * self.ang_mom**2 / (2. * self.mu * r**3)
        
    def Ueff_deriv(self, r):
        """dU_eff/dr"""
        return self.U_deriv(r) + self.Ucf_deriv(r)
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dr/dt d^2r/dt^2 dphi/dt]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            3-component vector with y[0] = r(t), y[1] = dr/dt, y[2] = dphi/dt
            
        """
        return [ y[1], 
                -1./self.mu * self.Ueff_deriv(y[0]), 
                self.ang_mom / (self.mu * y[0]**2) ]
    
    
    def solve_ode(self, t_pts, r_0, r_dot_0, phi_0, 
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        For now use odeint, but we have the option to switch.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [r_0, r_dot_0, phi_0]  
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        r, r_dot, phi = solution.y
        return r, r_dot, phi
    
    def energy(self, t_pts, r, r_dot):
        """Evaluate the energy as a function of time"""
        return (self.mu/2.) * r_dot**2 + self.Ueff(r)

In [3]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

# Gravity: $n = -1$

In [ ]:
n = 2
k = 1. 
ang_mom = 2. 
o1 = Orbit(ang_mom, n=n, k=k, mu=1)

fig_2 = plt.figure(figsize=(14,15))
ax_2 = fig_2.add_subplot(3,1,1)

r_pts = np.linspace(0.001, 3., 200)
U_pts = o1.U(r_pts)
Ucf_pts = o1.Ucf(r_pts)
Ueff_pts = o1.Ueff(r_pts)

ax_2.plot(r_pts, U_pts, linestyle='dashed', color='blue', label='U(r)')
ax_2.plot(r_pts, Ucf_pts, linestyle='dotted', color='green', label='Ucf(r)')
ax_2.plot(r_pts, Ueff_pts, linestyle='solid', color='red', label='Ueff(r)')

ax_2.set_xlim(0., 3.)
ax_2.set_ylim(-5., 5.)
ax_2.set_xlabel('r')
ax_2.set_ylabel('U(r)')
ax_2.set_title(f'$n = {n},\ \ k = {k},\ \  l = {ang_mom}$')
ax_2.legend(loc='lower right')
ax_2.axhline(0., color='black', alpha=0.3)

n2 = -1
k2 = -1.  
o2 = Orbit(ang_mom, n=n2, k=k2, mu=1)

ax_3 = fig_2.add_subplot(3,1,2)

U_pts2 = o2.U(r_pts)
Ucf_pts2 = o2.Ucf(r_pts)
Ueff_pts2 = o2.Ueff(r_pts)

ax_3.plot(r_pts, U_pts2, linestyle='dashed', color='blue', label='U(r)')
ax_3.plot(r_pts, Ucf_pts2, linestyle='dotted', color='green', label='Ucf(r)')
ax_3.plot(r_pts, Ueff_pts2, linestyle='solid', color='red', label='Ueff(r)')

ax_3.set_xlim(0., 3.)
ax_3.set_ylim(-5., 5.)
ax_3.set_xlabel('r')
ax_3.set_ylabel('U(r)')
ax_3.set_title(f'$n = {n2},\ \ k = {k2},\ \  l = {ang_mom}$')
ax_3.legend(loc='lower right')

ax_3.axhline(0., color='black', alpha=0.3)

n3 = -3
k3 = -1.  
o3 = Orbit(ang_mom, n=n3, k=k3, mu=1)

ax_4 = fig_2.add_subplot(3,1,3)

U_pts3 = o3.U(r_pts)
Ucf_pts3 = o3.Ucf(r_pts)
Ueff_pts3 = o3.Ueff(r_pts)

ax_4.plot(r_pts, U_pts3, linestyle='dashed', color='black', label='U(r)')
ax_4.plot(r_pts, Ucf_pts3, linestyle='dotted', color='green', label='Ucf(r)')
ax_4.plot(r_pts, Ueff_pts3, linestyle='solid', color='red', label='Ueff(r)')

ax_4.set_xlim(0., 3.)
ax_4.set_ylim(-5., 5.)
ax_4.set_xlabel('r')
ax_4.set_ylabel('U(r)')
ax_4.set_title(f'$n = {n3},\ \ k = {k3},\ \  l = {ang_mom}$')
ax_4.legend(loc='lower right')

ax_4.axhline(0., color='black', alpha=0.3)

fig_2.tight_layout()

fig_2.savefig('Taylor_8_14a.png',bbox_inches='tight')


In [4]:
t_start = 0.
t_end = 10.
delta_t = .0005
r_pts = np.linspace(0.001, 3., 20000)
n = 2
k = 1. 
ang_mom = 2. 
o1 = Orbit(ang_mom, n=n, k=k, mu=1)

U1 = o1.U(r_pts)
t_pts1 = np.arange(t_start, t_end, delta_t)

T1 = o1.energy(t_pts=t_pts1, r=r_pts, r_dot=0) - U1

print(T1.shape, t_pts1.shape, r_pts.shape)
T_avg = simps(T1, t_pts1, delta_t)/(t_end)
U_avg = simps(o1.Ueff(r_pts), t_pts1, delta_t)/(t_end)

delta_E1 = T_avg - o1.n * U_avg / 2
print(delta_E1)





(20000,) (20000,) (20000,)
-3.0008502833168222


## Orbit (time dependence)

We'll directly solve the equations for r(t) and phi(t).

In [ ]:
n4 = 2
k4 = 1.  
o4 = Orbit(ang_mom, n=n4, k=k4, mu=1)

n5 = -1
k5 = -1.  
o5 = Orbit(ang_mom, n=n5, k=k5, mu=1)

n6 = 7
k6 = 1.  
o6 = Orbit(ang_mom, n=n6, k=k6, mu=1)

# Plotting time 
t_start = 0.
t_end = 50.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Initial conditions
r_01 = (ang_mom**2/(k4*n4))**(1/(n4+2))
r_dot_0 = 0.
phi_0 = 0.0
r_pts, r_dot_pts, phi_pts = o4.solve_ode(t_pts, r_01, r_dot_0, phi_0)
r_ptse, r_dot_ptse, phi_ptse = o4.solve_ode(t_pts, r_01+.02, r_dot_0, phi_0)

r_02 = (ang_mom**2/(k5*n5))**(1/(n5+2))

r_pts2, r_dot_pts2, phi_pts2 = o5.solve_ode(t_pts, r_02, r_dot_0, phi_0)
r_ptse2, r_dot_ptse2, phi_ptse2 = o5.solve_ode(t_pts, r_02+.02, r_dot_0, phi_0)

r_03 = (ang_mom**2/(k6*n6))**(1/(n6+2))

r_pts3, r_dot_pts3, phi_pts3 = o6.solve_ode(t_pts, r_03, r_dot_0, phi_0)
r_ptse3, r_dot_ptse3, phi_ptse3 = o6.solve_ode(t_pts, r_03+.02, r_dot_0, phi_0)

c = o4.ang_mom**2 / (np.abs(o4.k) * o4.mu)
epsilon = c / r_0 - 1.
energy_0 = o4.mu/2. * r_dot_0**2 + o4.Ueff(r_0)
print(f'energy = {energy_0:.2f}')
print(f'eccentricity = {epsilon:.2f}')

In [ ]:
fig_4 = plt.figure(figsize=(15,15))


#ax_4a = fig_4.add_subplot(2,2,1)
#ax_4a.plot(t_pts, r_pts, color='black')
#ax_4a.set_xlabel(r'$t$')
#ax_4a.set_ylabel(r'$r$')
#ax_4a.set_title('Time dependence of radius')

#ax_4b = fig_4.add_subplot(2,2,2)
#ax_4b.plot(t_pts, phi_pts/(2.*np.pi), color='black')
#ax_4b.plot(t_pts, phi_pts/(2.*np.pi)%1, color='red')
#ax_4b.set_xlabel(r'$t$')
#ax_4b.set_ylabel(r'$\phi/2\pi$')
#ax_4b.set_title(r'Time dependence of $\phi$')

ax_4c = fig_4.add_subplot(3,1,1)
ax_4c.plot(r_pts*np.cos(phi_pts), r_pts*np.sin(phi_pts), color='black')
ax_4c.plot(r_ptse*np.cos(phi_ptse), r_ptse*np.sin(phi_ptse), color='magenta')
ax_4c.set_xlabel(r'$x$')
ax_4c.set_ylabel(r'$y$')
ax_4c.set_aspect(1)
ax_4c.set_xlim(-2,2)

ax_5c = fig_4.add_subplot(3,1,2)
ax_5c.plot(r_pts2*np.cos(phi_pts2), r_pts2*np.sin(phi_pts2), color='black')
ax_5c.plot(r_ptse2*np.cos(phi_ptse2), r_ptse2*np.sin(phi_ptse2), color='magenta')
ax_5c.set_xlabel(r'$x$')
ax_5c.set_ylabel(r'$y$')
ax_5c.set_aspect(1)
ax_5c.set_xlim(-5,5)

ax_6c = fig_4.add_subplot(3,1,3)
ax_6c.plot(r_pts3*np.cos(phi_pts3), r_pts3*np.sin(phi_pts3), color='black')
ax_6c.plot(r_ptse3*np.cos(phi_ptse3), r_ptse3*np.sin(phi_ptse3), color='magenta')
ax_6c.set_xlabel(r'$x$')
ax_6c.set_ylabel(r'$y$')
ax_6c.set_aspect(1)
ax_6c.set_xlim(-2,2)


fig_4.tight_layout()

fig_4.savefig('Taylor_8.14c.png',bbox_inches='tight')

## Energy versus time

In [ ]:
E_tot_pts = o1.energy(t_pts, r_pts, r_dot_pts)
E_tot_0 = E_tot_pts[0]
E_tot_rel_pts = np.abs((E_tot_pts - E_tot_0)/E_tot_0)

print(f'    t        E_tot        rel. error')
for t, E_tot, E_tot_rel in zip(t_pts, E_tot_pts, E_tot_rel_pts):
    print(f'{t:8.5f}  {E_tot:8.5e}  {E_tot_rel:12.4e}')

In [ ]:
fig_5 = plt.figure(figsize=(6,6))

overall_title = 'Gravitational orbit:  ' + \
                rf' $n = {o1.n},$' + \
                rf' $k = {o1.k:.1f},$' + \
                rf' $l = {o1.ang_mom:.1f},$' + \
                rf' $r_0 = {r_0:.1f},$' + \
                rf' $\dot r_0 = {r_dot_0:.2f},$' + \
                rf' $\phi_0 = {phi_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig_5.suptitle(overall_title, va='baseline')

ax_5a = fig_5.add_subplot(1,1,1)
ax_5a.semilogy(t_pts, np.abs(E_tot_pts), color='black', label=r'$E(t)$')
ax_5a.semilogy(t_pts, E_tot_rel_pts, color='red', label=r'$\Delta E(t)$')
ax_5a.set_xlabel(r'$t$')
ax_5a.set_ylabel(r'Energy')
ax_5a.set_title('Change in energy with time')
ax_5a.legend(loc='center right')

fig_5.tight_layout()